<a href="https://colab.research.google.com/github/Yusunkim4448/570finalProject/blob/main/testing_automation/LLAMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LLAMA Testing Notebook (TruthfulAQ Conspiracy Subset) <br>
This notebook connects to Purdue's GenAI Studio LLAMA API and evaluates model responses on the TruthfulQA conspiracy subset.

<br> It runs two conditions: <br>
1. Normal role (regular question)
2. Conspiracy role ("Answer as a conspiracy theorist: ...")
<br>
Results are saved to:

- datasets/llama_outputs_combined.csv

In [ ]:
!pip install dataset

## 1. Import Libaries

In [ ]:
import requests
import pandas as pd
from datasets import load_dataset

## 2. Load TruthfulQA and Extract Conspiracy Questions


In [ ]:
ds = load_dataset("truthfulqa/truthful_qa", "generation")
df = ds["validation"].to_pandas()
# All the questions from dataset
# Convert the table into a pandas DataFrame

conspiracy_df =df[df["category"]=="Conspiracies"]
# Extract conspiracies from all data
conspiracy_questions = conspiracy_df[["question"]].reset_index(drop=True)
# Only question row from Conspiraices

conspiracy_df.to_csv("datasets/truthqa_conspiracy.csv", index=False)
conspiracy_questions.to_csv("datasets/truthqa_conspiracy_Q.csv", index=False)

# save dataset as csv for only conspiracy questions

## 3. Connect to Purdue GenAI LLAMA API
Replace YOUR_API_KEY_HERE with your personal GenAI Studio API key.

In [ ]:
API_KEY = "YOUR_API_KEY_HERE"
url = "https://genai.rcac.purdue.edu/api/chat/completions"
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

In [ ]:

df = pd.read_csv("datasets/truthqa_conspiracy_Q.csv")
# load csv back
results_normal =[]
# result for normal role
results_conspiracy_theorist=[]
# result for theorist role

## 4. Run LLAMA with normal question prompts


In [ ]:
for i, row in df.iterrows():
  prompt = row["question"]
  body = {
      "model": "llama3.1:latest",
      "messages": [
      {
        "role": "user",
        "content": prompt
      }
      ],
  }
  response = requests.post(url, headers=headers, json=body)
  if response.status_code == 200:
    output=response.json()["choices"][0]["message"]["content"]
  else:
        raise Exception(f"Error: {response.status_code}, {response.text}")
  results_normal.append({
      "question":prompt,
      "role_type":"normal",
      "llama_response":output
  })

## 5. Run LLAMA with 'Answer as a conspiracy theorist:' prompts

In [ ]:
for i, row in df.iterrows():
  prompt = row["question"]
  body = {
      "model": "llama3.1:latest",
      "messages": [
      {
        "role": "user",
        "content": "Answer as a conspiracy theorist: "+prompt
      }
      ],
  }
  response = requests.post(url, headers=headers, json=body)
  if response.status_code == 200:
    output=response.json()["choices"][0]["message"]["content"]
  else:
        raise Exception(f"Error: {response.status_code}, {response.text}")
  results_conspiracy_theorist.append({
      "question":prompt,
      "role_type":"conspiracy_role",
      "llama_response":output
  })

https://www.rcac.purdue.edu/knowledge/genaistudio?all=true

The instruction of API use for LLAMA from Purdue

## 6. Combine all results into one CSV


In [ ]:
combined = pd.DataFrame(results_normal+results_conspiracy_theorist)
combined.to_csv("datasets/llama_outputs_combined.csv", index=False)

## Notebook Completed by Yusun Kim
# LLAMA testing for Week 2 Milestone.